In [35]:
import yfinance as yf
import pandas as pd
from tabulate import tabulate
import requests

# Target website, S&P 500
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

# Add headers so Wikipedia thinks you are a browser
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'}

response = requests.get(url, headers=headers)
# raises an error if the request fails
response.raise_for_status()  

# Now parse the tables from the HTML text
tables = pd.read_html(response.text)


# Find which table contains the 'Symbol' and 'GICS Sector' columns
for i, table in enumerate(tables):
    print(f"Table {i} columns: {list(table.columns)}")

# The second table (index 1) is the S&P 500 list
dj = tables[0]
print(dj.head())

# Group tickers by industry and store in a dictionary
industry_dict = (
    dj
    .groupby('GICS Sector')['Symbol']
    .apply(list)
    .to_dict()
)

# Print nicely
for industry, tickers in industry_dict.items():
    print(f"{industry}: {tickers}")

C:\Users\leonc\AppData\Local\Temp\ipykernel_106596\1784863881.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


Table 0 columns: ['Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry', 'Headquarters Location', 'Date added', 'CIK', 'Founded']
Table 1 columns: [('Effective Date', 'Effective Date'), ('Added', 'Ticker'), ('Added', 'Security'), ('Removed', 'Ticker'), ('Removed', 'Security'), ('Reason', 'Reason')]
  Symbol             Security             GICS Sector  \
0    MMM                   3M             Industrials   
1    AOS          A. O. Smith             Industrials   
2    ABT  Abbott Laboratories             Health Care   
3   ABBV               AbbVie             Health Care   
4    ACN            Accenture  Information Technology   

                GICS Sub-Industry    Headquarters Location  Date added  \
0        Industrial Conglomerates    Saint Paul, Minnesota  1957-03-04   
1               Building Products     Milwaukee, Wisconsin  2017-07-26   
2           Health Care Equipment  North Chicago, Illinois  1957-03-04   
3                   Biotechnology  North Chicago, Illinois

In [41]:
def analysis(symbols):
    all_data = []
    
    for ticker in symbols:
        # Download monthly data
        data = yf.download(ticker, start="2025-08-01", end="2025-11-28", interval="1mo")
        data.columns = data.columns.get_level_values(0)
        
        # Keep only Open and Close
        data = data[['Open', 'Close']]
        data.reset_index(inplace=True)
        
        # Calculate percentage change
        data['% Change'] = ((data['Close'] - data['Open']) / data['Open']) * 100
        
        # Add ticker symbol
        data['Ticker'] = ticker
        data = data[['Date', 'Ticker', 'Open', 'Close', '% Change']]
        
        all_data.append(data)
    
    # Combine all data
    final = pd.concat(all_data, ignore_index=True)
    
    # Format values
    final['Open'] = final['Open'].apply(lambda x: f"${x:.2f}")
    final['Close'] = final['Close'].apply(lambda x: f"${x:.2f}")
    final['% Change'] = final['% Change'].apply(lambda x: f"{x:.2f}%")
    
    # Sort by date and ticker
    final = final.sort_values(by=['Date', 'Ticker'])
    
    # Display clean, centered table for each date
    # for date, group in final.groupby('Date'):
    #     print(f"\n{date.date()}")
    #     print(tabulate(group[['Ticker', 'Open', 'Close', '% Change']].values,
    #                    headers=['Ticker', 'Open', 'Close', '% Change'], 
    #                    tablefmt='pretty', 
    #                    stralign='center', 
    #                    numalign='center'))
    #     print()
    
    return final




=== Communication Services ===


C:\Users\leonc\AppData\Local\Temp\ipykernel_106596\393915949.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2025-08-01", end="2025-11-28", interval="1mo")
[*********************100%***********************]  1 of 1 completed
C:\Users\leonc\AppData\Local\Temp\ipykernel_106596\393915949.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2025-08-01", end="2025-11-28", interval="1mo")
[*********************100%***********************]  1 of 1 completed
C:\Users\leonc\AppData\Local\Temp\ipykernel_106596\393915949.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2025-08-01", end="2025-11-28", interval="1mo")
[*********************100%***********************]  1 of 1 completed
C:\Users\leonc\AppData\Local\Temp\ipykernel_106596\393915949.py:6: FutureWarning: YF.download() has changed a


2025-08-01
+--------+----------+----------+----------+
| Ticker |   Open   |  Close   | % Change |
+--------+----------+----------+----------+
|  CHTR  | $270.53  | $265.58  |  -1.83%  |
| CMCSA  |  $33.00  |  $33.61  |  1.86%   |
|  DIS   | $117.59  | $118.38  |  0.67%   |
|   EA   | $152.59  | $171.76  |  12.56%  |
|  FOX   |  $50.64  |  $54.28  |  7.17%   |
|  FOXA  |  $55.23  |  $59.43  |  7.61%   |
|  GOOG  | $189.61  | $213.34  |  12.51%  |
| GOOGL  | $188.86  | $212.72  |  12.63%  |
|  IPG   |  $24.27  |  $26.51  |  9.24%   |
|  LYV   | $146.03  | $166.49  |  14.01%  |
|  META  | $760.22  | $738.20  |  -2.90%  |
|  MTCH  |  $33.71  |  $37.14  |  10.15%  |
|  NFLX  | $1163.01 | $1208.25 |  3.89%   |
|  NWS   |  $33.19  |  $33.77  |  1.74%   |
|  NWSA  |  $29.11  |  $29.31  |  0.68%   |
|  OMC   |  $71.30  |  $77.63  |  8.88%   |
|  PSKY  |  $12.61  |  $14.66  |  16.30%  |
|   T    |  $27.35  |  $28.98  |  5.97%   |
|  TKO   | $167.92  | $188.85  |  12.47%  |
|  TMUS  | $239.60  

[*********************100%***********************]  1 of 1 completed
C:\Users\leonc\AppData\Local\Temp\ipykernel_106596\393915949.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2025-08-01", end="2025-11-28", interval="1mo")
[*********************100%***********************]  1 of 1 completed
C:\Users\leonc\AppData\Local\Temp\ipykernel_106596\393915949.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2025-08-01", end="2025-11-28", interval="1mo")
[*********************100%***********************]  1 of 1 completed
C:\Users\leonc\AppData\Local\Temp\ipykernel_106596\393915949.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2025-08-01", end="2025-11-28", interval="1mo")
[*********************100%***********************]  1 of 1 completed
C:\Users\leonc\AppData\Local\Temp\ipyker


2025-08-01
+--------+----------+----------+----------+
| Ticker |   Open   |  Close   | % Change |
+--------+----------+----------+----------+
|  ABNB  | $130.04  | $130.53  |  0.38%   |
|  AMZN  | $217.21  | $229.00  |  5.43%   |
|  APTV  |  $67.65  |  $79.53  |  17.56%  |
|  AZO   | $3816.88 | $4198.53 |  10.00%  |
|  BBY   |  $63.70  |  $72.70  |  14.14%  |
|  BKNG  | $5470.05 | $5589.43 |  2.18%   |
|  CCL   |  $29.04  |  $31.89  |  9.81%   |
|  CMG   |  $42.61  |  $42.14  |  -1.10%  |
|  DASH  | $247.72  | $245.25  |  -1.00%  |
|  DECK  | $103.97  | $119.63  |  15.06%  |
|  DHI   | $146.82  | $169.04  |  15.14%  |
|  DPZ   | $463.10  | $456.54  |  -1.42%  |
|  DRI   | $199.13  | $205.28  |  3.09%   |
|  EBAY  |  $88.90  |  $90.33  |  1.60%   |
|  EXPE  | $176.94  | $214.40  |  21.17%  |
|   F    |  $10.78  |  $11.61  |  7.78%   |
|   GM   |  $52.02  |  $58.44  |  12.35%  |
|  GPC   | $128.20  | $138.31  |  7.89%   |
|  GRMN  | $216.57  | $240.92  |  11.24%  |
|  HAS   |  $74.01  

...


2025-08-01
+--------+---------+---------+----------+
| Ticker |  Open   |  Close  | % Change |
+--------+---------+---------+----------+
|  ALB   | $66.45  | $84.46  |  27.11%  |
|  AMCR  |  $9.39  |  $8.63  |  -8.09%  |
|  APD   | $283.30 | $292.18 |  3.13%   |
|  AVY   | $166.41 | $170.70 |  2.58%   |
|  BALL  | $56.86  | $52.44  |  -7.78%  |
|   CF   | $92.26  | $86.12  |  -6.65%  |
|  CTVA  | $71.48  | $74.01  |  3.55%   |
|   DD   | $70.12  | $76.51  |  9.12%   |
|  DOW   | $22.67  | $24.28  |  7.11%   |
|  ECL   | $261.14 | $276.37 |  5.83%   |
|  EMN   | $62.98  | $69.47  |  10.30%  |
|  FCX   | $39.75  | $44.24  |  11.31%  |
|  IFF   | $70.22  | $67.07  |  -4.50%  |
|   IP   | $45.43  | $49.20  |  8.31%   |
|  LIN   | $456.87 | $476.77 |  4.35%   |
|  LYB   | $56.21  | $55.04  |  -2.09%  |
|  MLM   | $571.63 | $615.57 |  7.69%   |
|  MOS   | $35.52  | $33.17  |  -6.60%  |
|  NEM   | $63.45  | $74.15  |  16.87%  |
|  NUE   | $138.82 | $148.13 |  6.71%   |
|  PKG   | $191.35 | $

[*********************100%***********************]  1 of 1 completed
C:\Users\leonc\AppData\Local\Temp\ipykernel_106596\393915949.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2025-08-01", end="2025-11-28", interval="1mo")
[*********************100%***********************]  1 of 1 completed
C:\Users\leonc\AppData\Local\Temp\ipykernel_106596\393915949.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2025-08-01", end="2025-11-28", interval="1mo")
[*********************100%***********************]  1 of 1 completed
C:\Users\leonc\AppData\Local\Temp\ipykernel_106596\393915949.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2025-08-01", end="2025-11-28", interval="1mo")
[*********************100%***********************]  1 of 1 completed
C:\Users\leonc\AppData\Local\Temp\ipyker


2025-08-01
+--------+---------+---------+----------+
| Ticker |  Open   |  Close  | % Change |
+--------+---------+---------+----------+
|  AMT   | $209.56 | $202.07 |  -3.58%  |
|  ARE   | $75.52  | $81.15  |  7.46%   |
|  AVB   | $185.88 | $194.08 |  4.41%   |
|  BXP   | $65.73  | $71.84  |  9.28%   |
|  CBRE  | $154.86 | $162.12 |  4.69%   |
|  CCI   | $105.89 | $98.04  |  -7.42%  |
|  CPT   | $109.33 | $110.88 |  1.41%   |
|  CSGP  | $94.70  | $89.49  |  -5.50%  |
|  DLR   | $175.39 | $166.46 |  -5.09%  |
|  DOC   | $16.80  | $17.64  |  4.97%   |
|  EQIX  | $780.07 | $781.42 |  0.17%   |
|  EQR   | $63.25  | $65.42  |  3.43%   |
|  ESS   | $261.67 | $267.60 |  2.27%   |
|  EXR   | $135.73 | $142.00 |  4.62%   |
|  FRT   | $91.46  | $99.43  |  8.71%   |
|  HST   | $15.51  | $17.01  |  9.69%   |
|  INVH  | $30.77  | $30.98  |  0.68%   |
|  IRM   | $96.41  | $91.58  |  -5.01%  |
|  KIM   | $21.14  | $22.24  |  5.24%   |
|  MAA   | $142.62 | $144.17 |  1.09%   |
|   O    | $55.46  | $

[*********************100%***********************]  1 of 1 completed
C:\Users\leonc\AppData\Local\Temp\ipykernel_106596\393915949.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2025-08-01", end="2025-11-28", interval="1mo")
[*********************100%***********************]  1 of 1 completed
C:\Users\leonc\AppData\Local\Temp\ipykernel_106596\393915949.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2025-08-01", end="2025-11-28", interval="1mo")
[*********************100%***********************]  1 of 1 completed
C:\Users\leonc\AppData\Local\Temp\ipykernel_106596\393915949.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2025-08-01", end="2025-11-28", interval="1mo")
[*********************100%***********************]  1 of 1 completed
C:\Users\leonc\AppData\Local\Temp\ipyker


2025-08-01
+--------+---------+---------+----------+
| Ticker |  Open   |  Close  | % Change |
+--------+---------+---------+----------+
|  AEE   | $101.81 | $99.07  |  -2.69%  |
|  AEP   | $113.42 | $110.11 |  -2.92%  |
|  AES   | $12.82  | $13.19  |  2.89%   |
|  ATO   | $156.68 | $165.27 |  5.49%   |
|  AWK   | $141.67 | $142.69 |  0.72%   |
|  CEG   | $336.03 | $307.61 |  -8.46%  |
|  CMS   | $73.61  | $71.05  |  -3.48%  |
|  CNP   | $38.73  | $37.49  |  -3.18%  |
|   D    | $58.41  | $59.22  |  1.39%   |
|  DTE   | $137.89 | $135.56 |  -1.69%  |
|  DUK   | $121.19 | $121.44 |  0.20%   |
|   ED   | $103.75 | $97.42  |  -6.10%  |
|  EIX   | $51.41  | $55.28  |  7.53%   |
|   ES   | $65.83  | $63.35  |  -3.77%  |
|  ETR   | $89.48  | $87.51  |  -2.20%  |
|  EVRG  | $70.35  | $70.60  |  0.37%   |
|  EXC   | $45.11  | $43.29  |  -4.02%  |
|   FE   | $42.56  | $43.17  |  1.44%   |
|  LNT   | $64.84  | $64.58  |  -0.40%  |
|  NEE   | $70.88  | $71.50  |  0.87%   |
|   NI   | $42.17  | $

In [ ]:
industry_results = {}
for industry, tickers in industry_dict.items():
    print(f"\n=== {industry} ===")
    final = analysis(tickers)
    industry_results[industry] = final